In [2]:
import numpy as np
import pandas as pd

csv_file_path = '../data/KHM/cambodia_dengue/DengueNet_Cases_Cambodia_1998_2010.csv'

In [3]:
df = pd.read_csv(csv_file_path, header=None, sep=",")
df.head()

,0,1,2,3,4,5,6,7,8,9,...,129,130,131,132,133,134,135,136,137,138
0,COUNTRY_ISO2_ID,COUNTRY_NAME,ADM_LVL1_ID,ADM_LVL1_NAME,1998 month: 1,1998 month: 2,1998 month: 3,1998 month: 4,1998 month: 5,1998 month: 6,...,2009 month: 12,2010 month: 1,2010 month: 2,2010 month: 3,2010 month: 4,2010 month: 5,2010 month: 6,2010 month: 7,2010 month: 8,2010 month: 9
1,KHM,Cambodia,KH.OM,Bântéay Méanchey,5,12,22,7,13,167,...,8,18,8,21,10,46,103,222,207,145
2,KHM,Cambodia,KH.BA,Batdâmbâng,13,26,23,33,57,136,...,3,3,3,6,5,9,14,54,75,30
3,KHM,Cambodia,KH.KM,Kâmpóng Cham,25,38,25,34,133,246,...,10,16,10,15,36,97,180,377,305,205
4,KHM,Cambodia,KH.KG,Kâmpóng Chhnang,7,5,2,11,36,105,...,1,1,1,4,6,27,38,77,97,49


In [4]:
def convert_date(raw_date):
    import re
    result = None
    date_regex = r'(\d{4})\s+month:\s+(\d{1,2})'
    matches = re.search(date_regex, raw_date)
    
    if matches:
        year = matches.group(1)
        month = matches.group(2)
        if len(month) == 1:
            month = '0' + month
        result = '{0}{1}01'.format(year, month)
    
    return result

In [11]:
arr = df.as_matrix().T

dates_raw = arr[4:, 0]
dates = [convert_date(x) for x in dates_raw]

places_raw = arr[:4, :]

new_arr = np.array([])

for i in range(1, arr.shape[1]):
    place_data = arr[:4, i]
       
    dengue_data = arr[4:, i]
    for j in range(dengue_data.shape[0]):
        date = dates[j]
        dengue_cases = dengue_data[j]
        if dengue_cases == 'n/a':
            dengue_cases = -1
        
        new_row = np.array(place_data)
        new_row = np.append(new_row, date)
        new_row = np.append(new_row, dengue_cases)
        
        if new_arr.shape[0] == 0:
            new_arr = new_row
        else:
            new_arr = np.vstack((new_arr, new_row))

        
column_names = ['country_iso2_id', 'country_name', 'admin_lvl1_id', 'admin_lvl1_name', 'period_date', 'dengue_total_cases']
new_df = pd.DataFrame(new_arr, columns=column_names)
new_df = new_df.apply(lambda x: pd.to_numeric(x, errors='ignore'))

new_df.head()

,country_iso2_id,country_name,admin_lvl1_id,admin_lvl1_name,period_date,dengue_total_cases
0,KHM,Cambodia,KH.OM,Bântéay Méanchey,19980101,5
1,KHM,Cambodia,KH.OM,Bântéay Méanchey,19980201,12
2,KHM,Cambodia,KH.OM,Bântéay Méanchey,19980301,22
3,KHM,Cambodia,KH.OM,Bântéay Méanchey,19980401,7
4,KHM,Cambodia,KH.OM,Bântéay Méanchey,19980501,13


In [12]:
new_df.to_csv('khm_denguenet_data_refactored.csv', index=False, header=True)